Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/

/content


裝環境

In [3]:
# Install PyTorch and torchvision
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

# Install bitsandbytes, accelerate, and transformers using pip
!pip install bitsandbytes==0.44.1
!pip install accelerate==1.0.1
!pip install transformers==4.45.2

# Install package language-evaluation
#!pip install git+https://github.com/bckim92/language-evaluation.git
#!python -c "import language_evaluation; language_evaluation.download('coco')"

# Install package CLIP
#!pip install ftfy regex tqdm
#!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 969.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 82.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.13.1+cu116 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import library

In [4]:
import os
import json
from PIL import Image
from tqdm import tqdm
import sys
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration


select an image and set parameters

In [37]:
# ------------- Parameters -------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_name = "ski"

Load the model and create processor

In [6]:
# Load model and processor
model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        output_attentions=True,
        #attn_implementation="eager"
).to(device)
processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

Load an image

In [38]:
image_path = os.path.join("/content/drive/MyDrive/Colab_Pro/DLCV/HW3/P2/hw3_data/p3_data/images", f"{image_name}.jpg")
image = Image.open(image_path)

Process image and prompt

In [39]:
prompt = "USER: <image>\nA short image caption ASSISTANT:"
# Process image and prompt
inputs = processor(images=image, text=prompt, return_tensors='pt').to(device, torch.float16)

In [40]:
# generate the response
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        do_sample=False,
        max_new_tokens=25,
        num_beams=3,
        use_cache=True,
        return_dict_in_generate=True,
        output_attentions=True,
    )

text = processor.decode(outputs["sequences"][0]).strip()
print(text)
output_sequence = outputs["sequences"][0]
print("output_sequence: \n", output_sequence)

<s> USER: <image> 
A short image caption ASSISTANT: A man and a woman are skiing on a snowy slope.</s>
output_sequence: 
 tensor([    1,  3148,  1001, 29901, 29871, 32000, 29871,    13, 29909,  3273,
         1967,  5777,   683,   319,  1799,  9047, 13566, 29901,   319,   767,
          322,   263,  6114,   526, 14993,   292,   373,   263, 15007, 29891,
        24968, 29889,     2], device='cuda:0')


process output_sequence

In [41]:
start_output_idx = 0
for idx, value in enumerate(output_sequence):
  if(value == 13566 and output_sequence[idx+1]== 29901): # !!! depends on input prompt!  # 13566->ANT, 29901->:
    start_output_idx = idx + 2
    break

print(start_output_idx)
output_sequence_split = output_sequence[start_output_idx:]
print("output_sequence_split: ", output_sequence_split)

18
output_sequence_split:  tensor([  319,   767,   322,   263,  6114,   526, 14993,   292,   373,   263,
        15007, 29891, 24968, 29889,     2], device='cuda:0')


get output attention

In [42]:
attentions = outputs.attentions
image_output_dir = f"/content/drive/MyDrive/Colab_Pro/DLCV/HW3/P3/{image_name}"
!mkdir {image_output_dir}

mkdir: cannot create directory ‘/content/drive/MyDrive/Colab_Pro/DLCV/HW3/P3/ski’: File exists


plot combined attention map

In [43]:
# Define the total number of tokens (including <start> and <endoftext>)
num_tokens = len(output_sequence_split) + 1  # +1 for the original image

# Set up grid size based on the total number of tokens (adjust as needed)
cols = 5  # Number of columns
rows = (num_tokens // cols) + (num_tokens % cols > 0)

plt.figure(figsize=(15, 10))

# Step 1: Plot the original image at position (0, 0)
plt.subplot(rows, cols, 1)
plt.title("<start>")
plt.axis('off')
original_image_resized = image.resize((224, 224))
original_image_np = np.array(original_image_resized) / 255.0  # Normalize to [0, 1]
plt.imshow(original_image_np)

# Step 2: Save attention maps for each token
for token_idx, token in enumerate(output_sequence_split):
    token_str = processor.decode([token])
    print(token_idx, token_str)

    # Access the attention weights from the last decoder block
    layer_idx = -1
    attention_map = attentions[token_idx + 1][layer_idx][0, :].mean(dim=0).cpu().numpy()
    attention_map = attention_map[0]  # (594, )

    # Take attention to image patches
    attn_map_to_image_patches = attention_map[1:577].copy()
    attn_map_to_image_patches_tensor = torch.tensor(attn_map_to_image_patches.reshape(24, 24), dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    attention_resized = F.interpolate(attn_map_to_image_patches_tensor, size=(224, 224), mode="bilinear", align_corners=False)
    attention_resized_np = attention_resized.squeeze().detach().cpu().numpy()

    attention_resized_np = np.clip(attention_resized_np**0.05, 0, 1)


    # Plot each token's attention map starting from position (0, 1)
    plt.subplot(rows, cols, token_idx + 2)  # +2 to account for the original image at (0, 0)
    plt.title(token_str)
    plt.axis('off')
    plt.imshow(original_image_np)
    plt.imshow(attention_resized_np, cmap='jet', alpha=0.5)

# Adjust layout and display the combined image
plt.tight_layout()

# Save the combined image
combined_save_path = os.path.join(image_output_dir, "combined_attention.png")
plt.savefig(combined_save_path, bbox_inches='tight', pad_inches=0)
plt.close()
print(f"Saved combined image to {combined_save_path}")

0 A
1 man
2 and
3 a
4 woman
5 are
6 ski
7 ing
8 on
9 a
10 snow
11 y
12 slope
13 .
14 </s>
Saved combined image to /content/drive/MyDrive/Colab_Pro/DLCV/HW3/P3/ski/combined_attention.png
